**Demo for `teneva.core.svd`**

---

Module contains the basic implementation of the TT-SVD algorithm as well as new original TT-SVD-incomplete algorithm, which implements efficient construction of the TT-tensor based on specially selected elements. This module also contains functions for constructing the SVD decomposition and skeleton decomposition for the matrices.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `matrix_skeleton`

Construct truncated skeleton decomposition A = U V for the given matrix.

In [2]:
# Shape of the matrix:
m, n = 100, 30

# Build random matrix, which has rank 3,
# as a sum of rank-1 matrices:
A = np.outer(np.random.randn(m), np.random.randn(n))
A += np.outer(np.random.randn(m), np.random.randn(n)) 
A += np.outer(np.random.randn(m), np.random.randn(n))

In [3]:
# Compute skeleton decomp.:
U, V = teneva.matrix_skeleton(A, e=1.E-10)

# Approximation error
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)

print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 30)
Error      : 9.00e-16


In [4]:
# Compute skeleton decomp with small rank:
U, V = teneva.matrix_skeleton(A, r=2)

# Approximation error:
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 2)
Shape of V : (2, 30)
Error      : 4.60e-01


## Function `matrix_svd`

Construct truncated SVD decomposition A = U V for the given matrix.

In [5]:
# Shape of the matrix:
m, n = 100, 30

# Build random matrix, which has rank 3,
# as a sum of rank-1 matrices:
A = np.outer(np.random.randn(m), np.random.randn(n))
A += np.outer(np.random.randn(m), np.random.randn(n)) 
A += np.outer(np.random.randn(m), np.random.randn(n))

In [6]:
# Compute SVD-decomp.:
U, V = teneva.matrix_svd(A, e=1.E-10)

# Approximation error:
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)

print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 17)
Shape of V : (17, 30)
Error      : 6.73e-16


In [7]:
# Compute SVD-decomp.:
U, V = teneva.matrix_svd(A, r=3)

# Approximation error:
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)

print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 30)
Error      : 6.64e-16


In [8]:
# Compute SVD-decomp.:
U, V = teneva.matrix_svd(A, e=1.E-2)

# Approximation error:
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)

print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 30)
Error      : 6.64e-16


In [9]:
# Compute SVD-decomp.:
U, V = teneva.matrix_svd(A, r=2)

# Approximation error:
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)

print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 2)
Shape of V : (2, 30)
Error      : 4.37e-01


## Function `svd`

Construct TT-tensor from the given full tensor using TT-SVD algorithm.

In [10]:
d = 20              # Dimension number
t = np.arange(2**d) # Tensor will be 2^d

# Construct d-dim full array:
Z_full = np.cos(t).reshape([2] * d, order='F')

In [11]:
# Construct TT-tensor by TT-SVD:
Y = teneva.svd(Z_full)

# Convert it back to numpy to check result:
Y_full = teneva.full(Y)

# Compute error for TT-tensor vs full tensor:
e = np.linalg.norm(Y_full - Z_full)
e /= np.linalg.norm(Z_full)

In [12]:
print(f'Size (np) : {Z_full.size:-8d}')       # Size of original tensor
print(f'Size (tt) : {teneva.size(Y):-8d}')    # Size of the TT-tensor
print(f'Erank     : {teneva.erank(Y):-8.2f}') # Eff. rank of the TT-tensor
print(f'Error     : {e:-8.2e}')               # Rel. error for TT-tensor vs full tensor

Size (np) :  1048576
Size (tt) :      152
Erank     :     2.00
Error     : 1.84e-14


## Function `svd_matrix`

Construct QTT-matrix from the given full matrix using TT-SVD algorithm.

In [13]:
q = 10   # Matrix size factor
n = 2**q # Matrix mode size

# Construct some matrix:
Z_full = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        Z_full[i, j] = np.cos(i) * j**2

In [14]:
# Construct QTT-matrix / TT-tensor by TT-SVD:
Y = teneva.svd_matrix(Z_full, e=1.E-6)

# Convert it back to numpy to check result:
Y_full = teneva.full_matrix(Y)

# Compute error for QTT-matrix / TT-tensor vs full matrix:
e = np.linalg.norm(Y_full - Z_full)
e /= np.linalg.norm(Z_full)

In [15]:
print(f'Size (np) : {Z_full.size:-8d}')       # Size of original tensor
print(f'Size (tt) : {teneva.size(Y):-8d}')    # Size of the QTT-matrix
print(f'Erank     : {teneva.erank(Y):-8.2f}') # Eff. rank of the QTT-matrix
print(f'Error     : {e:-8.2e}')               # Rel. error for QTT-matrix vs full tensor

Size (np) :  1048576
Size (tt) :     1088
Erank     :     5.71
Error     : 3.64e-12


## Function `svd_incomplete`

Construct TT-tensor from the given specially selected samples.

In [16]:
d = 20              # Dimension number
n = [2] * d         # Shape of the tensor/grid
t = np.arange(2**d) # Tensor will be 2^d

# Construct d-dim full array:
Z_full = np.cos(t).reshape([2] * d, order='F')

In [17]:
m = 4 # The expected TT-rank

# Generate special samples (indices) for the tensor:
I_trn, idx, idx_many = teneva.sample_tt(n, m)

In [18]:
# Compute tensor values in I multiindices:
Y_trn = np.array([Z_full[tuple(i)] for i in I_trn])

In [19]:
Y = teneva.svd_incomplete(I_trn, Y_trn,
    idx, idx_many, e=1.E-10, r=3) # Construct TT-tensor
teneva.show(Y)                    # Show the tensor

TT-tensor    20D : |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2| |2|
<rank>  =    2.0 :   \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/


In [20]:
# Convert it back to numpy to check result:
Y_full = teneva.full(Y)                          

# Compute error for TT-tensor vs full tensor :
e = np.linalg.norm(Y_full - Z_full)
e /= np.linalg.norm(Z_full)

In [21]:
print(f'Size (np) : {Z_full.size:-8d}')       # Size of original tensor
print(f'Size (tt) : {teneva.size(Y):-8d}')    # Size of the TT-tensor
print(f'Erank     : {teneva.erank(Y):-8.2f}') # Eff. rank of the TT-tensor
print(f'Error     : {e:-8.2e}')               # Rel. error for TT-tensor vs full tensor

Size (np) :  1048576
Size (tt) :      152
Erank     :     2.00
Error     : 2.50e-15


---